In [ ]:
# Import necessary libraries
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest, VectorConfig
from transformers import AutoTokenizer

# Set environment variables
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Initialize LLM with steering vector capability
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/openai-community/gpt2/",
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1
)


In [ ]:
# Create a simple example prompt for testing
example = "How to spend our life?"
# Generate baseline response without steering
example_answers = llm.generate(
    example,
    SamplingParams(
        temperature=0,
        max_tokens=50,
        skip_special_tokens=False,
    ),
)

# Display baseline response
print("=====Baseline=====")
print(example_answers[0].outputs[0].text)

sv_request = SteerVectorRequest(
    steer_vector_name="lm_steer",
    steer_vector_id=1,
    steer_vector_local_path="gpt2.pt", # https://huggingface.co/spaces/Glaciohound/LM-Steer/blob/main/checkpoints/gpt2.pt
    prefill_trigger_positions=[-1],
    algorithm="lm_steer",
    scale=1.0, # here positive - Toxification, negative - Detoxification
    target_layers=[11] # last layer
)
# Generate response with lm_steer
output = llm.generate(
    example, 
    SamplingParams(
        temperature=0,
        max_tokens=50,
        skip_special_tokens=False,
    ), 
    steer_vector_request=sv_request
)

# Display lm_steer response
print("=====lm_steer=====")
print(output[0].outputs[0].text)